In [1]:
# DAY01
nums = [int(x) for x in open('data/01.txt', 'r').readlines()]

reg = {}

TOTAL = 2020
for i, n in enumerate(nums):
    rest = TOTAL - n
    if rest in reg:
        print(f'{n} * {rest} = {n * rest}')
    reg[n] = i
    
for i, n1 in enumerate(nums):
    for j in range(i + 1, len(nums)):
        n2 = nums[j]
        rest = TOTAL - n1 - n2
        if rest in reg and reg[rest] > j:
            print(f'{n1} * {n2} * {rest} = {n1 * n2 * rest}')

1564 * 456 = 713184
764 * 857 * 399 = 261244452


In [2]:
# DAY02
def parse_line(line):
    parts = line.strip().split()
    counts = list(map(int, parts[0].split('-')))
    return (counts[0], counts[1], parts[1][0], parts[2])

def is_valid1(desc):
    cnt1, cnt2, ch, pwd = desc
    cnt = pwd.count(ch)
    return cnt1 <= cnt <= cnt2

def is_valid2(desc):
    cnt1, cnt2, ch, pwd = desc
    return (pwd[cnt1 - 1] == ch) ^ (pwd[cnt2 - 1] == ch)
    
passwords = list(map(parse_line, open('data/02.txt', 'r').readlines()))
res1 = sum([is_valid1(pwd) for pwd in passwords])
res2 = sum([is_valid2(pwd) for pwd in passwords])

print(f'Number of valid passwords: {res1}, {res2}')

Number of valid passwords: 548, 502


In [3]:
# DAY03
import math 

def count_trees(rows, dx, dy):
    h = len(rows)
    if h == 0:
        return 0
    w = len(rows[0])
    x, y = 0, 0
    res = 0
    while y < h:
        res += rows[y][x % w] == '#'
        x += dx
        y += dy
    return res

rows = [x.strip() for x in open('data/03.txt', 'r').readlines()]
res1 = count_trees(rows, 3, 1)
print(f'Num trees 1: {res1}')

SLOPES = [(1, 1), (3, 1), (5, 1), (7, 1), (1, 2)]
res2 = math.prod((count_trees(rows, x, y) for (x, y) in SLOPES))
print(f'Num trees 2: {res2}')

Num trees 1: 242
Num trees 2: 2265549792


In [5]:
# DAY04
import re

def read_doc(line):
    return dict([x.split(':') for x in line.split()])

REQUIRED_FIELDS = ['byr', 'iyr', 'eyr', 'hgt', 'hcl', 'ecl', 'pid']

def is_valid1(passport):
    return all(f in passport for f in REQUIRED_FIELDS)

def in_range(int_str, minv, maxv):
    try:
        return minv <= int(int_str) <= maxv
    except:
        return False

def is_valid_height(height_str):
    match = re.match(r"^(?P<in>\d+)in|(?P<cm>\d+)cm$", height_str)
    if match == None:
        return False
    hcm, hin = match.group('cm'), match.group('in')
    if hcm != None:
        return in_range(hcm, 150, 193)
    elif hin != None:
        return in_range(hin, 59, 76)
    else:
        return False

VALID_EYE_COLORS = ['amb', 'blu', 'brn', 'gry', 'grn', 'hzl', 'oth']

def is_valid2(passport):
    return is_valid1(passport) \
        and in_range(passport['byr'], 1920, 2002) \
        and in_range(passport['iyr'], 2010, 2020) \
        and in_range(passport['eyr'], 2020, 2030) \
        and passport['ecl'] in VALID_EYE_COLORS \
        and re.match(r"^(\d{9})$", passport['pid']) is not None \
        and re.match(r"^#[0-9a-f]{6}$", passport['hcl']) is not None \
        and is_valid_height(passport['hgt'])
    
lines = open('data/04.txt', 'r').read()
passports = [read_doc(line) for line in lines.split("\n\n")]

print(f'Valid passports 1: {sum(map(is_valid1, passports))}')
print(f'Valid passports 2: {sum(map(is_valid2, passports))}')

Valid passports 1: 264
Valid passports 2: 224


In [6]:
# DAY05
from functools import reduce

def get_id(seat):
    return reduce(lambda res, c: res * 2 + (c in 'BR'), seat, 0)

lines = [x.strip() for x in open('data/05.txt', 'r').readlines()]
ids = [get_id(line) for line in lines]

idset = set(ids)
seats = [id for id in range(1 << len(lines[0]))
    if id not in idset and id + 1 in idset and id - 1 in idset]

print(f'Max ID: {max(ids)}, Seat: {seats[0]}')

Max ID: 801, Seat: 597


In [7]:
# DAY06
groups = [g.split() for g in open('data/06.txt', 'r').read().split('\n\n')]

res1, res2 = 0, 0
for group in groups:
    counts = {}
    for answers in group:
        for ans in answers:
            if ans in counts:
                counts[ans] += 1
            else: 
                counts[ans] = 1
    res1 += len(counts)
    res2 += sum(counts[k] == len(group) for k in counts.keys())

print(f'Answer 1: {res1}, Answer 2: {res2}')

Answer 1: 6310, Answer 2: 3193


In [8]:
# DAY07
def parse_mapping(line):
    parts = line.split()
    bag_color = ' '.join(parts[0:2])
    res = {}
    for i in range(4, len(parts), 4):
        if parts[i] != 'no':
            res[' '.join(parts[i + 1:i + 3])] = int(parts[i])
    return (bag_color, res)

def build_back_mapping(mapping):
    res = {}
    for (bag, children) in mapping.items():
        for child in children.keys():
            if child in res:
                res[child].add(bag)
            else:
                res[child] = set([bag])
    return res

def count_reachable(root, back_mapping):
    visited = set()
    def rec(node):
        if node in visited:
            return 0
        res = 1
        if node in back_mapping:
            for child in back_mapping[node]:
                res += rec(child)
        visited.add(node)
        return res
    return rec(root)
    
def count_contains(root, mapping):
    res = 1
    for (child, count) in mapping[root].items():
        res += count * count_contains(child, mapping)
    return res
    
ROOT = "shiny gold"

lines = [g.strip() for g in open('data/07.txt', 'r').readlines()]
mapping = dict(parse_mapping(line) for line in lines)
back_mapping = build_back_mapping(mapping)

print(f'Reachable: {count_reachable(ROOT, back_mapping) - 1}')
print(f'Contains: {count_contains(ROOT, mapping) - 1}')

Reachable: 370
Contains: 29547


In [9]:
# DAY08
import enum

class Op(enum.Enum): 
    acc = 1
    nop = 2
    jmp = 3
    
def parse_op(line):
    parts = line.split()
    return (Op[parts[0]], int(parts[1]))

def eval_ops(ops, acc=0):
    n = len(ops)
    visited = [False]*n
    ip = 0
    while ip >= 0 and ip < n:
        if visited[ip]:
            return (False, acc)
        visited[ip] = True

        (op, val) = ops[ip]
        if op == Op.jmp:
            ip += val - 1
        elif op == Op.acc:
            acc += val
        ip += 1
    return (True, acc)

def try_mutate_program(ops):
    for i, (op, val) in enumerate(ops):
        prevOp = op
        if op == Op.jmp:
            op = Op.nop
        elif op == Op.nop:
            op = Op.jmp
        else:
            continue
            
        ops[i] = (op, val)
        (terminated, acc) = eval_ops(ops)
        if terminated:
            return acc
        ops[i] = (prevOp, val)
    
ops = [parse_op(g.strip()) for g in open('data/08.txt', 'r').readlines()]

print(f'Answer 1: {eval_ops(ops)[1]}, Answer 2: {try_mutate_program(ops)}')

Answer 1: 1782, Answer 2: 797


In [10]:
# DAY09
import bisect

def get_first_non_sum(nums, k):
    window = sorted(nums[:k])
    for i in range(k, len(nums)):
        l, r = 0, k - 1
        s = nums[i]
        while l < r:
            cur_sum = window[l] + window[r]
            if cur_sum < s:
                l += 1
            elif cur_sum > s:
                r -= 1
            else: 
                break
        if l == r:
            return s
        bisect.insort(window, s)
        del window[bisect.bisect_left(window, nums[i - k])]
        
def find_sum_range(nums, s):
    l, r = 0, 0
    cur_sum = nums[0]
    n = len(nums)
    while r < n - 1 and l < n - 1:
        if cur_sum < s:
            r += 1
            cur_sum += nums[r]
        elif cur_sum > s:
            cur_sum -= nums[l]
            l += 1
        else:
            break
    if l != r:
        return (l, r)
        
nums = [int(g) for g in open('data/09.txt', 'r').readlines()]

WINDOW_SIZE = 25

non_sum = get_first_non_sum(nums, WINDOW_SIZE)
print(f'First non-sum: {non_sum}')

(b, e) = find_sum_range(nums, non_sum)
print(f'Sum range: {min(nums[b:e + 1]) + max(nums[b:e + 1])}')

First non-sum: 85848519
Sum range: 13414198


In [11]:
# DAY10
nums = sorted(int(g) for g in open('data/10.txt', 'r').readlines())
nums.append(nums[-1] + 3)
nums.insert(0, 0)

diffs = []
for i, n in enumerate(nums[1:]):
    diffs.append(n - nums[i])
    
res1 = sum(map(lambda x: x == 1, diffs)) * sum(map(lambda x: x == 3, diffs))
print(f'Sum 1/3 diffs: {res1}')

n = len(nums)
counts = [-1]*n

def count_arrangements(idx):
    if idx == n - 1:
        return 1
    if counts[idx] >= 0:
        return counts[idx]
    res = 0
    i = idx
    while i < n - 1:
        i += 1
        diff = nums[i] - nums[idx]
        if diff <= 3:
            res += count_arrangements(i)
        else:
            break        
    counts[idx] = res
    return res

res2 = count_arrangements(0)
print(f'Number of arrangements: {res2}')

Sum 1/3 diffs: 2414
Number of arrangements: 21156911906816


In [12]:
# DAY11

OFFS = [[-1, -1], [0, -1], [1, -1], [1, 0], [1, 1], [0, 1], [-1, 1], [-1, 0]]

def get_neighbor_counts(seats):
    h, w = len(seats), len(seats[0])
    res = [[0]*w for i in range(h)]
    for y in range(h):
        for x in range(w):
            n = 0
            for dx, dy in OFFS:
                x1, y1 = x + dx, y + dy
                n += x1 >= 0 and x1 < w and y1 >= 0 and y1 < h \
                    and seats[y1][x1] == '#'
            res[y][x] = n
    return res

def get_visible_neighbor_counts(seats):
    h, w = len(seats), len(seats[0])
    cansee = [[0]*w for i in range(h)]
    res = [[0]*w for i in range(h)]
    
    for dx, dy in OFFS:
        for j in range(h):
            for i in range(w):
                x = i if dx <= 0 else w - i - 1
                y = j if dy <= 0 else h - j - 1
                x1, y1 = x + dx, y + dy
                if x1 >= 0 and x1 < w and y1 >= 0 and y1 < h:
                    s = seats[y1][x1]
                    if s == '.':
                        cansee[y][x] = cansee[y1][x1]
                    elif s == '#':
                        cansee[y][x] = True
                    elif s == 'L':
                        cansee[y][x] = False
                else:
                    cansee[y][x] = False
                res[y][x] += cansee[y][x]
    return res
                
def step(seats, new_seats, neighbors_fn, max_occupied):
    neighbor_counts = neighbors_fn(seats)
    
    num_changed = 0
    total_occupied = 0
    for y in range(len(seats)):
        for x in range(len(seats[y])):
            status = seats[y][x]
            n = neighbor_counts[y][x]
            if status == 'L' and n == 0:
                status = '#'
            elif status == '#' and n >= max_occupied:
                status = 'L'
            
            new_seats[y][x] = status
            num_changed += status != seats[y][x]
            total_occupied += status == '#'
    return (num_changed, total_occupied)
                
def iter_seats(seats, neighbors_fn, max_occupied):
    s1 = [line.copy() for line in seats]
    s2 = [line.copy() for line in seats]
    s = [s1, s2]

    idx = 0
    while True:
        (changed, occ) = step(s[idx], s[1 - idx], neighbors_fn, max_occupied)
        if changed == 0:
            return occ
        idx = 1 - idx
    
seats = [list(line.strip()) for line in open('data/11.txt', 'r').readlines()]
print(f'Answer 1: {iter_seats(seats, get_neighbor_counts, 4)}')
print(f'Answer 2: {iter_seats(seats, get_visible_neighbor_counts, 5)}')

Answer 1: 2211
Answer 2: 1995


In [13]:
# DAY12
import math
from functools import reduce

OFFS = {
    'N': (0, -1),
    'E': (1, 0),
    'S': (0, 1),
    'W': (-1, 0),
}

def step1(pos, command):
    x, y, angle = pos
    c, val = command
    if c == 'F':
        c = {270: 'N', 0: 'E', 90: 'S', 180: 'W'}[angle]
    if c in OFFS:
        offs = OFFS[c]
        x += offs[0] * val
        y += offs[1] * val
    else:
        if c == 'R':
            angle += val
        elif c == 'L':
            angle -= val
        angle = (angle + 360) % 360
    return (x, y, angle)

def step2(pos, command):
    x, y, wx, wy = pos
    c, val = command
    if c in OFFS:
        offs = OFFS[c]
        wx += offs[0] * val
        wy += offs[1] * val
    elif c == 'F':
        x += wx * val
        y += wy * val
    elif c in ['R', 'L']:
        angle = val
        if c == 'L':
            angle = -angle
        ar = math.radians(angle)
        ca, sa = math.cos(ar), math.sin(ar)
        wx1 = ca * wx - sa * wy
        wy1 = sa * wx + ca * wy
        wx, wy = round(wx1), round(wy1)
    return (x, y, wx, wy)
  
commands = [(line[0], int(line[1:])) for line in open('data/12.txt', 'r').readlines()]

x1, y1, angle1 = reduce(step1, commands, (0, 0, 0))
print(f'Answer 1: {abs(x1) + abs(y1)}')

x2, y2, wx2, wy2 = reduce(step2, commands, (0, 0, 10, -1))
print(f'Answer 2: {abs(x2) + abs(y2)}')

Answer 1: 508
Answer 2: 30761


In [14]:
# DAY13
def part1(periods, ts):
    min_diff = None
    min_p = 0
    res = 0
    for p in periods:
        if p is None:
            continue
        d = p - ts % p
        if min_diff == None or d < min_diff:
            min_diff = d
            res = d * p
            min_p = p
    return res

def part2(periods):
    stops = sorted(((p, i) for (i, p) in enumerate(periods) if p is not None))
    p0, i0 = stops[0]
    step = p0
    k = 0
    ts = i0
    while True:
        p1, i1 = stops[k + 1]
        if (ts + i1) % p1 == 0:
            step *= p1
            k += 1
            if k == len(stops) - 1:
                return ts
        ts += step

lines = open('data/13.txt', 'r').readlines()
ts = int(lines[0])
periods = [int(x) if x.isdigit() else None for x in lines[1].strip().split(',')]

print(f'Answer 1: {part1(periods, ts)}')
print(f'Answer 2: {part2(periods)}')

Answer 1: 153
Answer 2: 471793476184394


In [15]:
# DAY14
import re
from collections import namedtuple

MemOp = namedtuple('MemOp', 'mask ops')

def parse_ops(lines):
    cur_op = None
    for line in lines:
        if line.startswith('mask'):
            if cur_op != None:
                yield cur_op
            cur_op = MemOp(re.search('mask = (.*)$', line).group(1), [])
        elif line.startswith('mem'):
            match = re.search('mem\[(\d+)\] = (\d+)', line)
            cur_op.ops.append((int(match.group(1)), int(match.group(2)))) 
    yield cur_op
            
def apply_mask1(val, mask):
    bit = 1
    for i in range(len(mask) - 1, -1, -1):
        c = mask[i]
        if c == '0':
            val = val & ~bit
        elif c == '1':
            val = val | bit
        bit = bit << 1
    return val

def eval1(ops):
    mem = {}
    for mask, ops in mem_ops:
        for (addr, val) in ops:
            mem[addr] = apply_mask1(val, mask)
    return mem

def apply_mask2(val, mask, mask_pos=0):
    if mask_pos == len(mask):
        yield val
        return
    bit = 1 << (len(mask) - mask_pos - 1)
    c = mask[mask_pos]

    if c == '0':
        yield from apply_mask2(val, mask, mask_pos + 1)
    if c == 'X':
        val = val & ~bit
        yield from apply_mask2(val, mask, mask_pos + 1)
    if c == '1' or c == 'X':
        val = val | bit
        yield from apply_mask2(val, mask, mask_pos + 1)        
    
def eval2(ops):
    mem = {}
    for mask, ops in mem_ops:
        for (addr, val) in ops:
            for masked_addr in apply_mask2(addr, mask):
                mem[masked_addr] = val
    return mem

lines = [line.strip() for line in open('data/14.txt', 'r').readlines()]
mem_ops = list(parse_ops(lines))

print(f'Answer 1: {sum(eval1(mem_ops).values())}')
print(f'Answer 2: {sum(eval2(mem_ops).values())}')

Answer 1: 8471403462063
Answer 2: 2667858637669


In [2]:
# DAY15
def eval(seed, stop_k):
    positions = {}
    next_num = 0
    k = 1
    while True:
        num = seed[k - 1] if k <= len(seed) else next_num
        if k == stop_k:
            return num

        if num in positions:
            next_num = k - positions[num]
        else: 
            next_num = 0
            
        positions[num] = k
        k += 1

seed = [int(s) for s in open('data/15.txt', 'r').read().split(',')]

print(f'Answer 1: {eval(seed, 2020)}')
print(f'Answer 2: {eval(seed, 30000000)}')

Answer 1: 412
Answer 2: 243


In [41]:
# DAY16
from collections import namedtuple

Notes = namedtuple('Notes', 'categories own_ticket tickets')

def parse_notes(data):
    def parse_category(line):
        parts = line.split(': ')
        ranges = [tuple(map(int, r.split('-'))) for r in parts[1].split(' or ')]
        return (parts[0], ranges)

    def parse_ticket(line):
        return [int (x) for x in line.split(',')]
    
    parts = [part.strip().split('\n') for part in data.split('\n\n')]
    categories = dict(parse_category(line) for line in parts[0])
    own_ticket = parse_ticket(parts[1][1])
    tickets = [parse_ticket(t) for t in parts[2][1:]]
    return Notes(categories, own_ticket, tickets)

def is_valid_category(cat_ranges, val):
    for r in cat_ranges:
        if r[0] <= val <= r[1]:
            return True
    return False

def get_invalid_entries(notes):
    for (i, t) in enumerate(notes.tickets):
        for val in t:
            valid = False
            for cat_ranges in notes.categories.values():
                if is_valid_category(cat_ranges, val):
                    valid = True
                    break
            if not valid:
                yield (i, val)
                
def get_valid_tickets(notes, invalid_entries):
    res = []
    invalid_tickets = set(x[0] for x in invalid_entries)
    for (i, t) in enumerate(notes.tickets):
        if i not in invalid_tickets:
            res.append(t)
    return res

def get_category_ordering(notes):
    cat_ranges = list(notes.categories.values())
    cat_names = list(notes.categories.keys())
            
    n = len(cat_names) 
    cands_left = n * n
    candidates = [set(range(n)) for i in range(n)]
    
    # sift out invalid candidates
    for t in notes.tickets:
        for (i, x) in enumerate(t):
            for cand in candidates[i].copy():
                if not is_valid_category(cat_ranges[cand], x):
                    candidates[i].remove(cand)
                    cands_left -= 1
            if cands_left == n:
                break
    
    # iteratively subtract single-element candidate sets from others
    while cands_left > n:
        for c in candidates:
            if len(c) == 1:
                val = next(iter(c))
                for (i, c1) in enumerate(candidates):
                    if len(c1) > 1 and val in c1:
                        candidates[i].remove(val)
                        cands_left -= 1

    return [cat_names[next(iter(x))] for x in candidates]

def part2(notes):
    ordering = get_category_ordering(valid_notes)

    res = 1
    for (i, cat_name) in enumerate(ordering):
        if cat_name.startswith('departure'):
            res *= notes.own_ticket[i]
    return res

notes = parse_notes(open('data/16.txt', 'r').read())

invalid_entries = list(get_invalid_entries(notes))
print(f'Answer 1: {sum(x[1] for x in invalid_entries)}')

valid_notes = Notes(notes.categories, notes.own_ticket, get_valid_tickets(notes, invalid_entries))
print(f'Answer 2: {part2(valid_notes)}')

Answer 1: 25916
Answer 2: 2564529489989


In [6]:
# DAY17
import itertools

def count_neighbors(field, pos):
    if len(field) == 0:
        return 0
    res = 0
    for p in itertools.product(*[range(x - 1, x + 2) for x in pos]):
        if p != pos and p in field:
            res += 1
    return res

def get_bounds(field):
    if len(field) == 0:
        return None
    val = next(iter(field))
    res = [[x, x] for x in val]
    for pos in field:
        for i, x in enumerate(pos):
            res[i][0] = min(res[i][0], x)
            res[i][1] = max(res[i][1], x)
    return res

def step(field):
    bounds = get_bounds(field)
    res = set()
    for pos in itertools.product(*[range(b[0] - 1, b[1] + 2) for b in bounds]):
        n = count_neighbors(field, pos)
        if pos in field:
            if n == 2 or n == 3:
                res.add(pos)
        else:
            if n == 3:
                res.add(pos)
    return res

def init_field(seed, ndim=3):
    field = set()
    for y in range(len(seed)):
        for x in range(len(seed[y])):
            if seed[y][x] == '#':
                field.add(tuple([x, y] + [0] * (ndim - 2)))
    return field

NUM_STEPS = 6

def run(seed, ndim, num_steps=NUM_STEPS):
    field = init_field(seed, ndim)
    for i in range(num_steps):
        field = step(field)
    return len(field)

seed = [line.strip() for line in open('data/17.txt', 'r').readlines()]
print(f'Answer 1: {run(seed, 3)}')
print(f'Answer 2: {run(seed, 4)}')

Answer 1: 223
Answer 2: 1884


In [2]:
# DAY18
ADD = 1
MUL = 2

def apply_op(op, a, b):
    if op == None:
        return a if b == None else b
    if op == ADD:
        return a + b
    elif op == MUL:
        return a * b
    
def parse_num(exp, pos):
    res = None
    while pos < len(exp) and '0' <= exp[pos] <= '9':
        d = ord(exp[pos]) - ord('0')
        res = res * 10 + d if res != None else d
        pos += 1
    return res, pos

def eval_exp(exp, pos=0, add_precedence=False):
    res = None
    op = None
    muls = []
    while pos < len(exp):
        c = exp[pos]
        if c == ')':
            pos += 1
            break
        elif c == '(':
            val, pos = eval_exp(exp, pos + 1, add_precedence)
            res = apply_op(op, res, val)
            op = None
        elif c == ' ':
            pos += 1
        elif c == '*':
            if add_precedence:
                muls.append(res)
                res = None
            else: 
                op = MUL
            pos += 1
        elif c == '+':
            op = ADD
            pos += 1
        else:
            val, pos = parse_num(exp, pos)
            if val != None:
                res = apply_op(op, res, val)
                op = None
    for mul in muls:
        res *= mul
    return res, pos
    
expressions = [line.strip() for line in open('data/18.txt', 'r').readlines()]
print(f'Answer 1: {sum(map(lambda x: eval_exp(x, 0, False)[0], expressions))}')
print(f'Answer 2: {sum(map(lambda x: eval_exp(x, 0, True)[0], expressions))}')

Answer 1: 45283905029161
Answer 2: 216975281211165


In [4]:
# DAY19
def parse_rule(line):
    parts = [s.strip() for s in line.split('|')]
    for part in parts:
        if part.startswith('"'):
            yield (True, part[1:-1])
        else:
            yield (False, part.split(' '))

def parse_input(data):
    chunks = list(map(lambda x: x.strip().split('\n'), data.split('\n\n')))
    rules = {}
    for line in chunks[0]:
        parts = line.split(': ')
        rules[parts[0]] = list(parse_rule(parts[1]))
    return (rules,  chunks[1])

def is_valid(rules, line):
    def match_seq(seq, line, pos):
        if len(seq) > 0:
            for cpos in match(rules[seq[0]], line, pos):
                yield from match_seq(seq[1:], line, cpos)
        else:
            yield pos
        
    def match(rule, line, pos):
        for leaf, body in rule:
            if leaf:
                if line[pos:].startswith(body):
                    yield pos + len(body)
            else:
                yield from match_seq(body, line, pos)
    matches = list(match(rules['0'], line, 0))
    return 0 if len(matches) == 0 else matches[0] == len(line)

def patch_rules(rules):
    rules['8'] = list(parse_rule('42 | 42 8'))
    rules['11'] = list(parse_rule('42 31 | 42 11 31'))
    
rules, strings = parse_input(open('data/19.txt', 'r').read())
print(f'Answer 1: {sum(is_valid(rules, s) for s in strings)}')

patch_rules(rules)
print(f'Answer 2: {sum(is_valid(rules, s) for s in strings)}')

Answer 1: 192
Answer 2: 296


In [23]:
### DAY20
import math

OPPOSITE_DIR = [2, 3, 0, 1]
OFFS =[(0, -1), (1, 0), (0, 1), (-1, 0)]
N = 10

def parse_tile(data):
    lines = [line.strip() for line in data.split('\n')]
    try:
        tile_id = int(lines[0].split()[1][:-1])
        return (tile_id, lines[1:])
    except:
        return None

def get_bit_mask(seq):
    res = 0
    for c in seq:
        res = (res << 1) | (c == '#')
    return res

def reverse_bits(val):
    res = 0
    for i in range(N):
        res = res << 1 | (val & 1)
        val = val >> 1
    return res

def get_borders(tile):
    (tile_id, lines) = tile
    top = get_bit_mask(lines[0])
    bottom = get_bit_mask(lines[-1])
    left = get_bit_mask(line[0] for line in lines)
    right = get_bit_mask(line[-1] for line in lines)
    return (top, right, bottom, left)
    
def rotate(borders):
    (t, r, b, l) = borders
    return (reverse_bits(l), t, reverse_bits(r), b)
    
def get_transforms(borders):
    for i in range(4):
        yield borders
        (t, r, b, l) = borders
        # flip horizontal/vertical
        yield (reverse_bits(t), l, reverse_bits(b), r)
        yield (b, reverse_bits(r), t, reverse_bits(l))
        borders = rotate(borders)
    pass

def create_mappings(tiles):
    mappings = [{}, {}, {} ,{}]
    for (i, tile) in enumerate(tiles):
        borders = get_borders(tile)
        for tm in set(get_transforms(borders)):
            for side in range(4):
                mask = tm[side]
                if mask not in mappings[side]:
                    mappings[side][mask] = []
                mappings[side][mask].append((i, tm))
    return mappings

def find_corners(tiles, mapping):
    res = []
    for (tile_idx, t) in enumerate(tiles):
        borders = get_borders(t)
        k = 0
        for i, mask in enumerate(borders):
            m = set(x[0] for x in mapping[OPPOSITE_DIR[i]][mask])
            if len(m) == 1 and tile_idx in m:
                k += 1
        if k == 2:
            res.append(tile_idx)
    return res

def get_layout(tiles):
    mappings = create_mappings(tiles)
    n = len(tiles)
    
    used = [False] * n
    num_used = 0
    res = {}
    
    def layout(tile_idx, orientation, x, y):
        nonlocal num_used        
        if used[tile_idx]:
            return False
        
        for (i, mask) in enumerate(orientation):
            x1, y1 = x + OFFS[i][0], y + OFFS[i][1]
            if (x1, y1) in res and res[(x1, y1)][1][OPPOSITE_DIR[i]] != mask:
                return False

        used[tile_idx] = True
        res[(x, y)] = (tile_idx, orientation)
        num_used += 1
        if num_used == n:
            return True
        
        found_fit = False
        for i, mask in enumerate(orientation):
            x1, y1 = x + OFFS[i][0], y + OFFS[i][1]
            if (x1, y1) in res:
                continue
            for (ci, co) in mappings[OPPOSITE_DIR[i]][mask]:
                if layout(ci, co, x1, y1):
                    found_fit = True
                    break
            if found_fit:
                break
        if not found_fit:
            used[tile_idx] = False
            num_used -= 1
            res.pop((x, y), None)
        return found_fit
    
    corners = find_corners(tiles, mappings)
    start_idx = corners[0]
    layout(start_idx, get_borders(tiles[start_idx]), 0, 0)
    return res

def build_image(tiles, layout):
    minx = min(x[0] for x in layout.keys())
    miny = min(x[1] for x in layout.keys())
    maxx = max(x[0] for x in layout.keys())
    maxy = max(x[1] for x in layout.keys())
    res = [['.' for j in range((maxx - minx + 1) * (N - 2))] for i in range((maxy - miny + 1) * (N - 2))]
    for pos, (tile_idx, b) in layout.items():
        borders = list(get_transforms(get_borders(tiles[tile_idx])))
        tm = borders.index(b)
        ang = tm // 3
        flip_x = (tm % 3) == 1
        flip_y = (tm % 3) == 2
        x = (pos[0] - minx) * (N - 2)
        y = (pos[1] - miny) * (N - 2)
        img = tiles[tile_idx][1]
        for i in range(N - 2):
            for j in range(N - 2):
                c = img[j][i]
                cx, cy = i + 1, j + 1
                if flip_x:
                    cx = N - cx - 1
                if flip_y:
                    cy = N - cy - 1
                    
                if ang == 1:
                    cx, cy = cy, N - cx - 1
                elif ang == 2:
                    cx, cy = N - cx - 1, N - cy - 1
                elif ang == 3:
                    cx, cy = N - cy - 1, cx

                res[y + j][x + i] = img[cy][cx]
    return [''.join(s) for s in res]

def count_hashes(image, pattern):
    w, h = len(image[0]), len(image)
    def matches_sub(x, y):
        checked = 0
        for j, p in enumerate(pattern):
            for i, c in enumerate(p):
                x1, y1 = x + i, y + j
                if x1 < 0 or y1 < 0 or x1 >= w or y1 >= h:
                    continue
                if pattern[j][i] == '#' and image[y1][x1] != '#':
                    return False
                checked += 1
        return checked > 0
    res = [list(line) for line in image]
    for y in range(h - len(pattern) + 1):
        for x in range(w - len(pattern[0]) + 1):
            if matches_sub(x, y):
                for j, p in enumerate(pattern):
                    for i, c in enumerate(p):
                        x1, y1 = x + i, y + j
                        if x1 < 0 or y1 < 0 or x1 >= w or y1 >= h:
                            continue
                        if pattern[j][i] == '#':
                            res[y + j][x + i] = 'O'
    num_o, num_x = 0, 0
    for y in res:
        for c in y:
            num_o += c == 'O'
            num_x += c == '#'

    return (num_o, num_x)

def rotate_img(image):
    res = [list(line) for line in image]
    n = len(image)
    for y in range(n):
        for x in range(n):
            res[y][x] = image[x][n - y - 1]
    return [''.join(s) for s in res]

def fliph(image):
    return [line[::-1] for line in image]

def flipw(image):
    return [line for line in image[::-1]]

def get_img_transforms(image):
    for i in range(4):
        yield image
        yield fliph(image)
        yield flipw(image)
        image = rotate_img(image)
        
def compute_roughness(tiles, pattern):
    layout = get_layout(tiles)
    image = build_image(tiles, layout)
    for img in get_img_transforms(image):
        (co, ch) = count_hashes(img, pattern)
        if co != 0:
            return ch
               
chunks = open('data/20.txt', 'r').read().split('\n\n')
tiles = [t for t in (parse_tile(chunk) for chunk in chunks) if t]            
res1 = math.prod(tiles[t][0] for t in find_corners(tiles, create_mappings(tiles)))
print(f'Answer 1: {res1}')

pattern = open('data/20.pattern.txt', 'r').read().split('\n')
res2 = compute_roughness(tiles, pattern)        
print(f'Answer 2: {res2}')

Answer 1: 63187742854073
Answer 2: 2152


In [3]:
# DAY21
def parse_rule(line):
    parts = line[:-1].split(' (contains ')
    return (set(parts[0].split(' ')), set(parts[1].split(', ')))

def get_mappings(rules):
    mappings = {}
    for i in range(len(rules)):
        c, a = rules[i]
        if len(c) == len(a) == 1:
            mappings[c.pop()] = a.pop()
            continue
        cxc, cxa = c, a
        for j in range(i + 1, len(rules)):
            c1, a1 = rules[j]
            if len(cxa & a1) > 0:
                cxc = cxc & c1
                cxa = cxa & a1
            if len(cxc) == len(cxa) == 1:
                mappings[cxc.pop()] = cxa.pop()
                break
    return mappings

def trim_rules(rules):
    res = {}
    while True:
        mappings = get_mappings(rules)
        if len(mappings) == 0:
            return res
        for (c, a) in mappings.items():
            res[c] = a
            for r in rules:
                if c in r[0]:
                    r[0].remove(c)
                if a in r[1]:
                    r[1].remove(a)

rules = [parse_rule(line.strip()) for line in open('data/21.txt', 'r').readlines()]
mappings = trim_rules(rules)
res1 = sum(len(r[0]) for r in rules)
print(f'Answer 1: {res1}')
      
res2 = ','.join(kv[0] for kv in sorted(mappings.items(), key=lambda x: x[1]))
print(f'Answer 2: {res2}')

Answer 1: 2374
Answer 2: fbtqkzc,jbbsjh,cpttmnv,ccrbr,tdmqcl,vnjxjg,nlph,mzqjxq


In [3]:
# DAY22
from collections import deque

def parse_deck(data):
    return deque(int(line.strip()) for line in data.split('\n')[1:])
    
def get_score(deck):
    return sum(x * (len(deck) - i) for (i, x) in enumerate(deck))

def step(decks, recurse):
    da, db = decks
    if len(db) == 0:
        return 0
    elif len(da) == 0:
        return 1
    a, b = da.popleft(), db.popleft()
    a_wins = False
    
    if recurse and a <= len(da) and b <= len(db):
        da1 = deque(da[i] for i in range(a))
        db1 = deque(db[i] for i in range(b))
        a_wins = game([da1, db1], True) == 0
    else:
        a_wins = a > b
        
    if a_wins:
        da.append(a)
        da.append(b)
    else:
        db.append(b)
        db.append(a)
    return -1

def game(decks, recurse):
    visited = [set(), set()]
    while True:
        h1, h2 = hash(str(decks[0])), hash(str(decks[1]))
        if h1 in visited[0] or h2 in visited[1]:
            # infinite loop, Player 1 wins
            return 0
        visited[0].add(h1)
        visited[1].add(h2)
        
        winner = step(decks, recurse)
        if winner != -1:
            return winner
    
decks = [parse_deck(chunk.strip()) for chunk in open('data/22.txt', 'r').read().split('\n\n')]

decks1 = [deck.copy() for deck in decks]
winner1 = game(decks1, False)
print(f'Answer 1: {get_score(decks1[winner1])}')

decks2 = [deck.copy() for deck in decks]
winner2 = game(decks2, True)
print(f'Answer 2: {get_score(decks2[winner2])}')


Answer 1: 31809
Answer 2: 32835


In [1]:
%%time
# DAY23
import itertools

class Node:
    def __init__(self, val):
        self.val = val
        self.next = None
        
class CircularList:
    def __init__(self, elems):
        self.lookup = {}
        self.current = None
        
        cur_node = None
        for el in elems:
            next_node = Node(el)
            if cur_node is None:
                self.current = next_node
                self.min_val = el
                self.max_val = el
            else:
                cur_node.next = next_node
                self.min_val = min(el, self.min_val)
                self.max_val = max(el, self.max_val)
            cur_node = next_node
            self.lookup[el] = cur_node
        cur_node.next = self.current
    
    def __repr__(self):
        node = self.current
        res = ""
        while True:
            res += f'({node.val}) ' if node == self.current else f'{node.val} '
            node = node.next
            if node == self.current:
                break
        return res
    
    def get_values(self, start_val=1):
        start_node = self.lookup[start_val]
        node = start_node.next
        while node != start_node:
            yield node.val
            node = node.next

def step(labels):
    to_move = []
    to_move_vals = []
    node = labels.current.next
    for i in range(3):
        to_move.append(node)
        to_move_vals.append(node.val)
        node = node.next
    labels.current.next = node
    
    dest = labels.current.val - 1
    while dest in to_move_vals or dest not in labels.lookup:
        dest -= 1
        if dest < labels.min_val:
            dest = labels.max_val
        
    dest_node = labels.lookup[dest]
    dnext = dest_node.next
    dest_node.next = to_move[0]
    to_move[-1].next = dnext
    
    labels.current = labels.current.next
    
def expand_seed(seed, total=1000000):
    val = seed[0]
    for x in seed:
        val = max(x, val)
        yield x
    for i in range(len(seed), total):
        val += 1
        yield val
        
def eval_game(labels, steps=100):
    for i in range(steps):
        if (i + 1) % 1000000 == 0:
            print(f'Step {i}')
        step(labels)

seed = [int(x) for x in list(open('data/23.txt', 'r').read().strip())]

labels1 = CircularList(seed)
eval_game(labels1, 100)
res1 = ''.join(str(n) for n in labels1.get_values())
print(f'Answer 1: {res1}')

labels2 = CircularList(expand_seed(seed))
eval_game(labels2, 10000000)

(a, b) = itertools.islice(labels2.get_values(), 2)
res2 = a *  b
print(f'Answer 2: {res2}')

Answer 1: 24798635
Step 999999
Step 1999999
Step 2999999
Step 3999999
Step 4999999
Step 5999999
Step 6999999
Step 7999999
Step 8999999
Step 9999999
Answer 2: 12757828710
CPU times: user 27.1 s, sys: 94.8 ms, total: 27.2 s
Wall time: 27.2 s


In [1]:
# DAY24
from functools import reduce

def parse_line(line):
    pos = 0
    while pos < len(line):
        if line[pos] in ['e', 'w']:
            yield line[pos]
            pos += 1
        elif line[pos:pos + 2] in ['se', 'sw', 'nw', 'ne']:
            yield line[pos:pos + 2]
            pos += 2
        else:
            return
        
def eval_step(pos, step):
    x, y = pos
    if len(step) == 1:
        x = x + 1 if step == 'e' else x - 1
    else:
        vdir, hdir = step[0], step[1]
        if y % 2 == 0:
            if hdir == 'w':
                x -= 1
        else:
            if hdir == 'e':
                x += 1
        y = y + 1 if vdir == 's' else y - 1
    return x, y
        
def get_neighbor_coord(pos):
    return (eval_step(pos, s) for s in 
            ['e', 'w', 'se', 'sw', 'nw', 'ne'])
        
def eval_path(steps):
    return reduce(eval_step, steps, (0, 0))

def eval_paths(paths):
    res = set()
    for path in paths:
        pos = eval_path(path)
        if pos in res:
            res.remove(pos)
        else:
            res.add(pos)
    return res

def step_day(tiles):
    res = set()
    minx = min(tiles, key=lambda p: p[0])[0]
    maxx = max(tiles, key=lambda p: p[0])[0]
    miny = min(tiles, key=lambda p: p[1])[1]
    maxy = max(tiles, key=lambda p: p[1])[1]
        
    for x in range(minx - 1, maxx + 2):
        for y in range(miny - 1, maxy + 2):
            t = x, y
            num_neighbors = sum(npos in tiles for npos in get_neighbor_coord(t))
            if t in tiles:
                if (num_neighbors == 0 or num_neighbors > 2):
                    # flip to white
                    pass
                else:
                    res.add(t)
            elif num_neighbors == 2:
                res.add(t)
    return res
            
paths = [list(parse_line(line)) 
         for line in open('data/24.txt', 'r').readlines()]

tiles = eval_paths(paths)
print(f'Answer 1: {len(tiles)}')

for i in range(100):
    tiles = step_day(tiles)
print(f'Answer 2: {len(tiles)}')

Answer 1: 469
Answer 2: 4353
